# Cleaning nutrional information data

In [ ]:
import pandas as pd

In [ ]:
# load the data
# nutrient information 
df_nutrients = pd.read_csv("../../data/food_csv/nutrient.csv")
# food name
df_food = pd.read_csv("../../data/food_csv/food.csv")
# food nutrient information
df_food_nutrient = pd.read_csv("../../data/food_csv/food_nutrient.csv")

In [ ]:
df_nutrients = df_nutrients.dropna()
df_nutrients = df_nutrients.astype({"nutrient_nbr": int})

Join tables to include all nutrients in a single df

In [ ]:
df_cleaned_nutrient = df_food_nutrient.merge(
    df_nutrients[["nutrient_nbr", "name", "unit_name"]],
    left_on="nutrient_id",
    right_on="nutrient_nbr",
    how="inner",
)
df_cleaned_nutrient = df_cleaned_nutrient[
    ["fdc_id", "nutrient_id", "amount", "name", "unit_name"]
]

Show the most common nutrients

In [ ]:
df_cleaned_nutrient[df_cleaned_nutrient["amount"] > 0].groupby(
    "name"
).count().sort_values(by="fdc_id", ascending=False)

Remove unwanted columns

In [ ]:
required_nutrients = (
    "Protein",
    "Energy",
    "Total lipid (fat)",
    "Carbohydrate, by summation",
    "Fiber, total dietary",
)
df_cleaned_nutrient = df_cleaned_nutrient[
    df_cleaned_nutrient["name"].isin(required_nutrients)
]

In [ ]:
df_cleaned_nutrient

Append the unit name to the end of the nutrient name, to simplify data structure

In [ ]:
df_cleaned_nutrient["nutrient"] = (
    df_cleaned_nutrient["name"] + " (" + df_cleaned_nutrient["unit_name"] + ")"
)

Pivot the table, turning each nutrient into a column, to be joined with the food table

In [ ]:
df_pivot_nutrient = df_cleaned_nutrient.pivot(
    index="fdc_id", columns="nutrient", values="amount"
)

In [ ]:
df_nutrional_information = df_food.merge(df_pivot_nutrient, on="fdc_id", how="left")

Rename columns to follow a standard and avoid whitespapce

In [ ]:
df_nutrional_information = df_nutrional_information.rename(
    columns={
        "description": "food_name",
        "Carbohydrate, by summation (G)": "carbohydrates_g",
        "Energy (KCAL)": "energy_kcal",
        "Total lipid (fat) (G)": "lipid_g",
        "Protein (G)": "protein_g",
        "Fiber, total dietary (G)": "fiber_g",
    },
)
df_nutrional_information = df_nutrional_information[
    ["food_name", "carbohydrates_g", "energy_kcal", "lipid_g", "protein_g", "fiber_g"]
]

Save the cleaned data

In [ ]:
df_nutrional_information = df_nutrional_information.dropna()
df_nutrional_information.to_csv("data/cleaned_food_data.csv", index=False)